# Regression

In this module, we will explore how to do regression analyses with R.

If you are unfamiliar with notebooks, please review some basics [here](https://github.com/michhar/useR2016-tutorial-jupyter). 

## Essential Tips

A very brief summary of the critical components and commands within jupyter are:

1. Critically, press `Ctrl+Enter` to run (or render) the current cell.
2. Output will print to the notebook. You may have to scroll up to see it all.
3. Get help for any function by typing a question mark and then its name into
   the console: `?rxLinMod`. It will split the window, and will bring up the documentation for 
   that function below.
5. Files will appear in the specified directory. You can find them by selecting File in the menu bar and selecting "Open...". This will open a new browser window with a file navigator.
6. R objects can be viewed by typing `ls()` in an R cell.
7. Run all the example code!

There are a number of hands-on exercises in the document, so while you can run the notebook from beginning to end, you will get a lot more out of it by actually walking through cell-by-cell, and filling out the corresponding exercises.

These notebooks are based on a tutorial presented at a Microsoft conference in June of 2016. The original files are available [here](https://github.com/joseph-rickert/MLADS_JUNE_2016).


## Regression Analysis

This module takes you through the process of fitting a multiple regression model. 

The data come from the UCI machine Learning Repository. The data are described here:
https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.names, and here: http://cs.nyu.edu/courses/fall00/G22.3033-001/weka/weka-3-0-2/data/auto-mpg.arff

### Setup  
Before we get started, we'll source a configuration file in the next cell. It simply makes sure that the relevant R packages and datasets are available. You do not need to look at it, but if you are interested, you can view the configuration file [here](Resources/config.R). It may take a few moments to run the first time you run it, but it should be fast afterwards.


In [ ]:
source("Resources/config.R")
source("Resources/splitData.R")

## Data 
Let's start by getting the data.

In [ ]:
data_dir <- 'data'
mpg_file <- file.path(data_dir,'mpg_orig.csv')
file.remove(mpg_file)
if(file.exists(mpg_file)){
    cat('*** Reading mpg file from local file.\n')
    mpg <- read.csv(mpg_file)
}else{
    cat('*** Reading mpg file from remote url.\n')
    url <-"https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
    mpg <- read.table(url,na.strings="?")
    names(mpg) <- c("mpg","cyl","disp","hp","weight","accel","year","origin","name")
    cat('*** Saving mpg file for posterity.\n')
    write.csv(mpg,file = mpg_file,row.names=FALSE)
    mpg <- read.csv(mpg_file, header = TRUE)
}
head(mpg)
dim(mpg)

## Data Exploration

Let's start by seeing how many missing values there are.

In [ ]:
sum(is.na(mpg))

## Let's do something about those

Let's remove the missing values since there are only six of them. In another context, you would potentially consider imputing the missing values.

In [ ]:
mpg <- na.omit(mpg)  ## remember this deletes the entire row!
dim(mpg)
sum(is.na(mpg))

## Summarize the data

Let's summarize the data first. First explore the class of each variable and then get univariate statistics for each variable.

In [ ]:
sapply(mpg,class)
summary(mpg)

## Coerce our variables

Next, it actually makes sense to create some categorical variables. For instance, we may care about the origin of the car as a categorical variable, we may want to use year as a categorical variable, and we may want to use cyl as a categorical variable. They are all numbers, but there is no strong reason to expect purely linear relationships between those variables and the outcome. 

In particular, if we look at the links above, we can also attach semantic labels to our origin variable.

In [ ]:
library(dplyr)
mpg <- mutate(mpg, cyl = as.factor(cyl),
              year = as.factor(year),
              origin = factor(origin, labels = c('USA','Europe','Japan')))
sapply(mpg,class)

### Exploratory Data Analysis
In this section we look at numerical summaries of the data as well as various plots to visualize relationships among the variables.

First, let's rerun the summary now that we have some categorical variables defined as such. We'll note that the factor variables end up producing frequency counts rather than arithmetic summaries.

In [ ]:
summary(mpg)

In [ ]:
help(dplyr::select)

In [ ]:
# Look at correlations among the numeric variables


In [ ]:
num_vars <- names(mpg)[sapply(mpg, is.numeric)]
num_vars
mpg_num <- select(mpg, one_of(num_vars))
cor(mpg_num)

In [ ]:
library(corrplot)
corrplot(cor(mpg_num), method="ellipse")

# Look at the at pairwise relationships for continuous variables

While `corrplot()` is an easy way to visually represent those relationships, we can also do some more sophisticated pieces with the `pairs()` function. It includes arguments where you can customize the panels to plot additional information.

THe `panel` argument in this case allows us to plot points and the best fitting OLS regression line, and the `diag.panel` argument allows us to simply provide a histogram of each variable.

In [ ]:
pairs(mpg[,num_vars],          
      panel=function(x,y){              
        points(x,y, col="light blue")   
        abline(lm(y~x), lty=2,col="red") 
      },
      diag.panel=function(x){           
        par(new=T)
        hist(x,main="",axes=F,nclass=12) 
      }
)

## Additional Graphics

As we investigate more, we can see that there are some non-linear relationships, and we can also explore how some of the numeric variables are related to some of the continuous ones. In order to examine the categorical variables, let's use ggplot

In [ ]:
library(ggplot2)
p1 <- ggplot(mpg, aes(x = cyl, y = mpg))
p1 + geom_boxplot()

In [ ]:
# mpg as a function of origin location
p2 <- ggplot(mpg, aes(x = origin, y = mpg))
p2 + geom_boxplot()

In [ ]:
# Let's color code according to origin, but plot two continous.
p3 <- ggplot(mpg, aes(x = weight, y = mpg,col=origin))
p3 + geom_point() + geom_smooth()

In [ ]:
# mpg by year
p4 <- ggplot(mpg, aes(x = year, y =mpg))
p4 + geom_boxplot()

## Fit Regression Models

In this section we divide randomly divide the data into training and test sets and then fit several regression models to the training data. Holding out some data enables us to assess the performance of these models by seeing how well they predict `mpg` for the test data. This kind of process is essential if you want to get an idea of how well the model will perform on data that it hasn't seen before. 

The function, `splitData()`, which is given above randomly splits the mpg data into two subsets: train and test. `mpg[ins$train,]` means index into the mpg data and return all of the columns of mpg but only the rows to be used for training the model.  

In [ ]:
set.seed(1)
ind <- splitData(mpg)  # Split the mpg data into training and test data.
lapply(ind, length)

Let's start by just using the numeric variables.

We can construct the model specification programmatically by creating a formula. In this case, we only want the numeric variables (contained within `num_vars`), and we only want to consider their main effects, so we construct a formula with `+` separating the predictor variables.

In [ ]:
form.1chr <- paste("mpg ~", paste(num_vars[-1], collapse = '+'))
form.1 <- formula(form.1chr)
form.1                  

In [ ]:
# Fit and summarize the model:
lm.fit.1  <- lm(formula=form.1, data=mpg[ind$train,])   # Build the model
summary(lm.fit.1)

## Model 2
Let's add the number of cylinders


In [ ]:
form.2chr <- paste(form.1chr, 'cyl', sep = '+')
form.2 <- formula(form.2chr)
form.2
lm.fit.2  <- lm(formula=form.2,data=mpg[ind$train,])   # Build the model
summary(lm.fit.2)

# Model 3

The third model looks at the effect of year. 

In [ ]:
form.3chr <- paste(form.2chr, 'year', sep = '+')
form.3 <- formula(form.3chr)
form.3
lm.fit.3  <- lm(formula=form.3,data=mpg[ind$train,])   # Build the model
summary(lm.fit.3)

## Model 4

The fourth model looks at the effect of origin (but excludes year)

In [ ]:
form.4chr <- paste(form.2chr, 'origin', sep = '+')
form.4 <- formula(form.4chr)
form.4
lm.fit.4  <- lm(formula=form.4,data=mpg[ind$train,])   # Build the model
summary(lm.fit.4)

## Model 5

This model includes cyl, year, and origin.


In [ ]:
form.5chr <- paste(form.4chr, 'year', sep = '+')
form.5 <- formula(form.5chr)
form.5
lm.fit.5  <- lm(formula=form.5,data=mpg[ind$train,])   # Build the model
summary(lm.fit.5)

## Model Selection

Model 5 had the best adjusted R squared, but it also had hte most degrees of freedom. We can use the `anova()` function to compare nested models.


In [ ]:
anova(lm.fit.1, lm.fit.2, lm.fit.3, lm.fit.5)
anova(lm.fit.1, lm.fit.2, lm.fit.4, lm.fit.5)

## Model Diagnostics

It looks like model 5 is the best, but we may want to evaluate the assumptions that are going into that inference. We also probably want to test on new data.

In this section we create the standard model diagnostic plots to evaluate how well the model fits the assumptions underlying regressin models. Look here http://www.stat.columbia.edu/~martin/W2024/R7.pdf for some information on interpreting the diagnostic plots. Except for a few outliers flagged by by the plotting software everything looks pretty good.

In [ ]:
# Plot the regression diagnostics
par(mfrow=c(2,2)) # set up device so all graphs are on one device 
c <- plot(lm.fit.5)
par(mfrow=c(1,1)) # move it back to one device

In [ ]:
# Look at the outliers flagged in the diagnostic plots
outliers <- c(273, 321, 326, 382 ,389)
mpg[outliers,]

### Assess Model Performance

Here we get an idea of how well the model will do on new data by using the predict function to "score" the new data set. We then plot the actual reported values of MPG against the values predicted by the regression with confidence intervals.

In [ ]:
predictions <- predict(lm.fit.5, newdata = mpg[ind$test,],se.fit=TRUE,interval="prediction")
str(predictions)


In [ ]:
# Create a data frame to hold the predictions
df <- data.frame(y = mpg[ind$test,]$mpg, predictions)
head(df,2)

## Now let's create some more plots

Let's plot predictions vs actuals in the test data set, where error bars correspond to the confidence interval provided by `predict()`, which by default is a 95% confidence interval.

In [ ]:
?predict.lm

In [ ]:
library(ggplot2)
p5 <- ggplot(df, aes(x = y, y = fit.fit))
p5 + geom_errorbar(aes(ymin = fit.lwr, ymax = fit.upr), width = .1) +
  geom_point() + 
  geom_abline(intercept = 0, slope = 1, linetype=2) +
  xlab("Reported MPG") +
  ylab("Predicted MPG") +
  ggtitle("95% CIs for Predictions")

This model looks pretty good! The 45 degree line, which indicates a perfect predictions, is mostly covered by the confidence intervals. 

For homework, try creating some more exploratory plots and building additional regression models. 